In [1]:
import time
import requests
from loguru import logger

### decorator

In [ ]:
import time
import requests
from loguru import logger
from functools import wraps


class MaxRetriesExceeded(Exception):
    pass


def with_exponential_backoff(max_retries=3, base_delay=1):
    """
    A decorator to apply exponential backoff to any function.
    """

    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            attempt = 1
            while True:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    if attempt == max_retries:
                        logger.warning(
                            f"{func.__name__} failed: {e}, reached max retries"
                        )
                        raise MaxRetriesExceeded(
                            f"Max retries {max_retries} exceeded for {func.__name__}"
                        )
                    else:
                        delay = base_delay * (2 ** (attempt - 1))
                        logger.warning(
                            f"{func.__name__} failed: {e}, will retry in {delay} seconds..."
                        )
                        time.sleep(delay)

                attempt += 1

        return wrapper

    return decorator

In [ ]:
# Example usage
@with_exponential_backoff(max_retries=3, base_delay=2)
def make_request(url):
    """
    Makes a request to the given URL.
    """
    response = requests.get(url)
    response.raise_for_status()
    return response


test_url = "https://httpbin.org/status/429"  # Simulates rate-limiting
try:
    response = make_request(test_url)
    print("Response:", response.text)
except MaxRetriesExceeded as e:
    logger.error(e)

2025-04-23 00:16:39.625 | WARNING  | __main__:wrapper:27 - make_request failed: 429 Client Error: TOO MANY REQUESTS for url: https://httpbin.org/status/429, will retry in 2 seconds...
2025-04-23 00:16:42.902 | WARNING  | __main__:wrapper:27 - make_request failed: 429 Client Error: TOO MANY REQUESTS for url: https://httpbin.org/status/429, will retry in 4 seconds...
2025-04-23 00:16:49.047 | WARNING  | __main__:wrapper:22 - make_request failed: 429 Client Error: TOO MANY REQUESTS for url: https://httpbin.org/status/429, reached max retries
2025-04-23 00:16:49.050 | ERROR    | __main__:<module>:18 - Max retries 3 exceeded for make_request


### non decorator example

In [ ]:
class MaxRetriesExceeded(Exception):
    pass


def make_request_with_backoff(url, max_retries=3, base_delay=1):
    """
    Makes a request to the given URL with exponential backoff on failure.
    """
    attempt = 1
    while True:
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as e:
            if attempt >= max_retries:
                logger.warning(
                    f"Request failed: {e}, Max retries {max_retries} exceeded."
                )
                raise MaxRetriesExceeded(f"Max retries exceeded for URL: {url}") from e
            else:
                delay = base_delay * (2 ** (attempt - 1))  # Exponential backoff
                logger.warning(f"Request failed: {e}, wait for {delay} seconds...")
                time.sleep(delay)

        attempt += 1

In [ ]:
test_url = "https://httpbin.org/status/429"  # Simulates rate-limiting
try:
    response = make_request_with_backoff(test_url, max_retries=3, base_delay=2)
except MaxRetriesExceeded as e:
    logger.error(e)

2025-04-22 23:06:15.105 | WARNING  | __main__:make_request_with_backoff:20 - Request failed: 429 Client Error: TOO MANY REQUESTS for url: https://httpbin.org/status/429, wait for 2 seconds...
2025-04-22 23:06:23.502 | WARNING  | __main__:make_request_with_backoff:20 - Request failed: 429 Client Error: TOO MANY REQUESTS for url: https://httpbin.org/status/429, wait for 4 seconds...
2025-04-22 23:06:29.751 | WARNING  | __main__:make_request_with_backoff:15 - Request failed: 429 Client Error: TOO MANY REQUESTS for url: https://httpbin.org/status/429, Max retries 3 exceeded.
2025-04-22 23:06:29.753 | ERROR    | __main__:<module>:5 - Max retries exceeded for URL: https://httpbin.org/status/429
